<a href="https://colab.research.google.com/github/michnotmeesh/murphys/blob/main/AprilTagDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PART 1**

In [ ]:
pip install apriltag

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for apriltag: filename=apriltag-0.0.16-cp310-cp310-linux_x86_64.whl size=511965 sha256=712cc507f1b851a25a3374156bf13e604d32a09efc8f9635ecfbe6b40bbc18d8
  Stored in directory: /root/.cache/pip/wheels/06/b0/0f/31eead0216b5f1df61d62733b48d7c2247912bfe64808f8ea5
Successfully built apriltag


In [ ]:
pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=5de01ebf05f9dcad3e0567b2b0fd8f499bc0d9fbe99ca16b141b70c37339bd45
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import csv
import apriltag
import datetime
import math
import numpy as np
import matplotlib.cm as cm
from matplotlib import pyplot as plt
from docx import Document
from docx.shared import Inches

In [ ]:
#TO DO:
# f
# K
# videoname
# cap
# tagsize
# directory
# options > quad_contours ~ F / T

In [ ]:
#default data
radius = 101.5 #mm
total_num_frame = 0

# camera parameters in pixels (final_)
fx = 0.7756362692955363 #220.82561105300002 mm
fy = 1.3514092147513248

cx = 1969.22358 # 3840/2 #-1.3085625174541973e-05
cy = 1126.711382 #2160/2 #-1.1451102520767267e-05
camera_params = (fx, fy,cx,cy)

#date and time
now = datetime.datetime.now()
date = now.strftime("_%m-%d-%Y_%H-%M-%S")

In [ ]:
##~~~ For the radius calculation
# moving thia inside ath e cap.isopened
# K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
# dist_coef = np.zeros((5,1))
# center = np.array([0, 0, 0])

###~~~ to contain tag positions on the OSA
tag_pos_osa = []

In [ ]:
#video file
videoname = "Flight1Parabola2Run1Cam5"
cap = cv2.VideoCapture('/content/drive/MyDrive/AR Tags/2. Data/Videos/'+ videoname + '.mp4') #change path name
i = 0

#artag file
artag_filename = videoname + date + '(0,0)'

In [ ]:
## get the Original width and height
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print('Width: ' + str(original_width))
print('Height: ' + str(original_height))

# Set the desired dimensions for downscaled frames
target_width = 1080 #1920
target_height = 960 #1080

# Calculate the scaling factors
scale_x = target_width / original_width
scale_y = target_height / original_height


In [ ]:
#directories
directory = "/content/drive/MyDrive/AR Tags/2. Data/Data/"
path = os.path.join(directory, artag_filename)
os.mkdir(path)
artag_dir = directory + artag_filename + "/"
frame_dir = "Frames"
path = os.path.join(artag_dir, frame_dir)
os.mkdir(path)
tag_dir = "Tags"
path = os.path.join(artag_dir, tag_dir)
os.mkdir(path)

In [ ]:
## timestamp
fps = cap.get(cv2.CAP_PROP_FPS)
print("FPS: ")
print(fps)
FPS= []
success, image = cap.read()
count = 0
success = True

In [ ]:

## CSV
fields = ['frame','time','tag ID','SROA/OSA','pos x', 'pos y', 'pos z',
        'R11','R12','R13','R21','R22','R23','R31','R32','R33',
        'XR(osa)', 'YR(osa)', 'ZR(osa)', 'OSA radius']
rows = []

filename = artag_dir + artag_filename + ".csv"


In [ ]:
with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    while success:
        while(cap.isOpened()):
            #ret, frame = cap.read()
            success, frame = cap.read()

            if success == False:
                break
        #naming each frame
            framename = artag_dir + frame_dir + '/' + artag_filename + '_frame_' + str(i)
            cv2.imwrite(framename + '.png', frame)

            img = framename + '.png'

            total_num_frame +=1

        #getting image
            image = cv2.imread(img)

        ### resize frame
            image = cv2.resize(image, None, fx=scale_x, fy=scale_y)

            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        #detecting tags
            print("[INFO] detecting AprilTags...(yessuh)")
            options = apriltag.DetectorOptions(families="tag36h11",
                    border =1, nthreads=4, quad_decimate =1.0,
                    quad_blur=0.0, refine_edges=True, refine_decode=False,
                    refine_pose=True, debug=True, quad_contours=True)
            detector = apriltag.Detector(options)
            results = detector.detect(gray)
            print("***[INFO] {} total AprilTags detected!".format(len(results)))
            # To check values of results
            print("**[INFO] Tags Detected: " + str(results) + " [END INFO]**")
            sroa = [111, 112, 113, 114, 14, 7, 8, 15] #can be seen in this cam5 angle
            osa =[]
            sroas=[]

        ## timestamp
            count+=1
            time= count/fps
            FPS.append(time)

            ###~~~ to contain tag positions on the OSA
            # tag_pos_osa = {}

        ## Moveed the  camera matrix and distortion coefficients array
            K = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
            dist_coef = np.zeros((5,1))

        # results
            for r in results:
                #print(r.decode('UTF-8'))
                (ptA, ptB, ptC, ptD) = r.corners
          #added for the TRIAL
                corners = r.corners

                ptA = (int(ptA[0]), int(ptA[1]))
                ptB = (int(ptB[0]), int(ptB[1]))
                ptC = (int(ptC[0]), int(ptC[1]))
                ptD = (int(ptD[0]), int(ptD[1]))
          # START changed image to frame
                cv2.line(frame, ptA, ptB, (0, 255, 0), 2)
                cv2.line(frame, ptB, ptC, (0, 255, 0), 2)
                cv2.line(frame, ptC, ptD, (0, 255, 0), 2)
                cv2.line(frame, ptD, ptA, (0, 255, 0), 2)

                (cX, cY) = (int(r.center[0]), int(r.center[1]))
                cv2.circle(frame, (cX,cY), 5, (0, 0, 255), -1)
                print("[INFO] Center: (" + str(r.center[0]) + ", " + str(r.center[1]) + ")")

                tagFamily = r.tag_family.decode("utf-8")
                tagID = str(r.tag_id)
                cv2.putText(frame, tagID, (ptA[0], ptA[1] - 15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                print("[INFO] tag family: {}".format(tagFamily))
                print("[INFO] tag ID: {}".format(tagID))

          # END changed image to frame
            #To clasify tags
                if r.tag_id in sroa:
                    print("[INFO] Tag {} is in SROA".format(str(r.tag_id)))
                    sroas.append(r.tag_id)
                    print('SROA: ')
                    print(sroas)
                    tag = "SROA"
                else:
                    print("[INFO] Tag {} is in OSA".format(str(r.tag_id)))
                    osa.append(r.tag_id)
                    print('OSA: ')
                    print(osa)
                    tag = "OSA"
                    tag_pos_osa.append(r)

                poses = detector.detection_pose(r, camera_params, tag_size=14, z_sign=1) #tagsize is 14 mm

                # rows.append([str(i), str(time), r.tag_id, tag,
                #              poses[0][0][3], poses[0][1][3], poses[0][2][3],
                #              poses[0][0][0], poses[0][0][1], poses[0][0][2],
                #              poses[0][1][0], poses[0][1][1], poses[0][1][2],
                #              poses[0][2][0], poses[0][2][1], poses[0][2][2],
                #              (poses[0][0][3] - (radius * poses[0][0][2])),
                #              (poses[0][1][3] - (radius * poses[0][1][2])),
                #              (poses[0][2][3] - (radius * poses[0][2][2]))])

            ####~~~~ to get the tvec and rvec
                # R = np.array([[poses[0][0][0], poses[0][0][1], poses[0][0][2]],
                #              [poses[0][1][0], poses[0][1][1], poses[0][1][2]],
                #              [poses[0][2][0], poses[0][2][1], poses[0][2][2]]])
                # t = np.array([poses[0][0][3], poses[0][1][3], poses[0][2][3]])
                # rvec, _ = cv2.Rodrigues(R)
                # tvec = t.reshape(-1,1)

## Trial from CHATGPT
                # Example usage of camera matrix K
                homogeneous_corners = np.vstack((corners.T, np.ones((1, 4))))  # Homogeneous coordinates
                world_corners = np.linalg.inv(K) @ homogeneous_corners  # Transform to world coordinates

                # Compute tag position and orientation
                _, rvec, tvec= cv2.solvePnP(world_corners.T, corners, K, dist_coef)
                rotation_matrix, _ = cv2.Rodrigues(rvec)
                pos = -np.dot(rotation_matrix.T, tvec)
                ori = rotation_matrix.T


                 # Print the detected tag information (in millimeters)
                print("Tag ID:", tagID)
                print("Position (in millimeters):", pos.flatten())
                print("Orientation (rotation matrix):", ori)
                print()
## END TRIAL


            #compute
                print("[START] THIS IS NOT FIXED")
                distance = np.linalg.norm(tvec)

                tag_size = np.mean(np.abs(r.corners[0]-r.corners[2]))
                r_pixel = tag_size * radius / (distance * np.mean([fx, fy])) # changing this to (fx,fy) => f)

                rad = r_pixel * radius / tag_size
                print("[END] THIS IS NOT FIXED")

                # tag_radius = get_sphere_radius()
        #has 1mm difference~~
                # # Calculate tag position relative to the camera
                # position = np.dot(R, np.array([0, 0, 0])) + t

                # # Calculate distance between camera and tag
                # distance = np.linalg.norm(position)

                # # Calculate radius of the sphere
                # tag_radius = distance - radius
        #~~


  ## OG
                # rows.append([str(i), str(time), r.tag_id, tag,
                #              poses[0][0][3], poses[0][1][3], poses[0][2][3],
                #              poses[0][0][0], poses[0][0][1], poses[0][0][2],
                #              poses[0][1][0], poses[0][1][1], poses[0][1][2],
                #              poses[0][2][0], poses[0][2][1], poses[0][2][2],
                #              (poses[0][0][3] - (radius * poses[0][0][2])),
                #              (poses[0][1][3] - (radius * poses[0][1][2])),
                #              (poses[0][2][3] - (radius * poses[0][2][2])),
                #              rad])
  ## END OG

                rows.append([str(i), str(time), r.tag_id, tag,
                             pos[0], pos[1], pos[2],
                             ori[0][0], ori[0][1], ori[0][2],
                             ori[1][0], ori[1][1], ori[1][2],
                             ori[2][0], ori[2][1], ori[2][2],
                             (ori[0][0] - (radius * ori[0][2])),
                             (ori[0][1] - (radius * ori[1][2])),
                             (ori[0][2] - (radius * ori[2][2])),
                             rad])




        ## Calculate sphere center and radius
                # if np.linalg.norm(tvec) < radius:
                #   tag_pos_osa[r.tag_id] = tvec.flatten()
            # if len(tag_pos_osa) >= 3:
            #   A = np.zeros((3, 3))
            #   B = np.zeros((3, 1))
            #   for tag_id in tag_pos_osa:
            #     A += np.outer(tag_pos_osa[tag_id], tag_pos_osa[tag_id])
            #     B += tag_pos_osa[tag_id].reshape(3, 1) * (radius ** 2 - np.linalg.norm(tag_pos_osa[tag_id]) ** 2)
            #   center = np.linalg.solve(A, B).flatten()
            #   radius = np.mean([np.linalg.norm(tag_pos_osa[tag_id]) for tag_id in tag_pos_osa])
            # print("**IMPORTANT**: Radius is: {}".format(radius))

            # rows.append([radius])

        #naming each frame
            tagname = artag_dir + tag_dir + '/' + artag_filename + '_tagFrame_' + str(i)
            # print("[Extract] It extracted "+ tagname)
            cv2.imwrite(tagname +'.png', image)
            csvwriter.writerows(rows)

            rows = []
            i+=1




In [ ]:
import shutil

# Deleting an non-empty folder
dir_path = artag_dir + frame_dir
shutil.rmtree(dir_path, ignore_errors=True)
print("Deleted '%s' directory successfully" % dir_path)

In [ ]:
# Deleting an non-empty folder
dir2_path = artag_dir + tag_dir
shutil.rmtree(dir2_path, ignore_errors=True)
print("Deleted '%s' directory successfully" % dir2_path)

In [ ]:
cap.release()
cv2.destroyAllWindows()

# **PART 2**
Plot the tags by:

*   All tags
*   Individual SROA tags
*   Individual OSA tags
*   SROA and OSA (2 distinctions only)






In [ ]:
def get_all_tags(get_filename):
    '''
    Returns a list of all detected tags, duplicated not allowed
    Given filename holds all the collected data from videos
    csv file has the following fields:
            frame, time, tag ID, SROA/OSA, pos x, pos y, pos z, R11,
            R12, R13, R21, R22, R23, R31, R32, R33
    '''
    tags = []
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
          tags.append(int(row['tag ID']))
    print(tags)
    tags = [*set(tags)]
    print(tags)
    return tags

In [ ]:
def get_pos_alt(get_tag):
    '''
    Returns a list of position arrays for a given tag and which frames
    that tag was detected. Will read in the csv file and build the position
    arrays (x, y, and z) for a given tag
    '''
    x = []
    y = []
    z = []
    frame = []

    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['tag ID'] == str(get_tag):
                x.append(float(row['pos x'].strip('[]')))
                y.append(float(row['pos y'].strip('[]')))
                z.append(float(row['pos z'].strip('[]')))
                frame.append(int(row['frame']))

    return x, y, z, frame


In [ ]:
################################
# Test get_all_tags function
# Need to upload the csv of the collected data in the same directory
# Set filename to the csv file

all_tags = get_all_tags(filename)
# print('all ')
# print(all_tags)
# print('filename ')
# print(filename)
# print('Amount of unique tags: ' + str(len(all_tags)))


In [ ]:
# Plot positions of ALL detected tags (OSA & SROA)
fig = plt.figure(figsize=(8, 8))

colors = cm.rainbow(np.linspace(0, 1, len(all_tags)))
#print(colors)
ax = plt.axes(projection='3d')
ax.set_title('Position of All Detected Tags')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Postion')
#print(range(len(all_tags)))

for i, c in zip(range(len(all_tags)), colors):
    curr_tag = all_tags[i]

    # print('curr_tag' + str(curr_tag))
    # print('all ' + str(len(all_tags)))
    # print('curr ' + str(curr_tag))
    x, y, z, frames = get_pos_alt(curr_tag)
    # print(len(x))
    # print('frames: ')
    # print(str(frames))
    # print('i: ')
    # print(i)
    # print('x: ' + str(x))

    ax.scatter(x, y, z, color=c, label=str(curr_tag))
    plt.legend(bbox_to_anchor=(1.2, 1), loc="upper left")
# plt.legend(loc="upper right")
plt.show()

In [ ]:
fig.savefig(artag_dir + artag_filename + 'all_tags.png')

In [ ]:
# Get only detected OSA tags from all tags data
osa_tags = []

for tag in all_tags:
  if tag < 111:
    osa_tags.append(tag)

# Plot positions of individual OSA tags only
fig = plt.figure(figsize=(8, 8))

colors = cm.rainbow(np.linspace(0, 1, len(osa_tags)))
ax = plt.axes(projection='3d')
ax.set_title('Position of Detected OSA Tags')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Postion')

for i, c in zip(range(len(osa_tags)), colors):
    curr_tag = osa_tags[i]

    x, y, z, frames = get_pos_alt(curr_tag)

    ax.scatter(x, y, z, color=c, label=str(curr_tag))
    plt.legend(bbox_to_anchor=(1.2, 1), loc="upper left")
plt.show()

In [ ]:
fig.savefig(artag_dir + artag_filename + 'osa_tags.png')

In [ ]:
# Get only detected SROA tags from all tags data
sroa_tags = []

for tag in all_tags:
  if tag > 110:
    sroa_tags.append(tag)

# Plot positions of individual SROA tags only
fig = plt.figure(figsize=(8, 8))

colors = cm.rainbow(np.linspace(0, 1, len(sroa_tags)))
ax = plt.axes(projection='3d')
ax.set_title('Position of Detected SROA Tags')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Postion')

for i, c in zip(range(len(sroa_tags)), colors):
    curr_tag = sroa_tags[i]

    x, y, z, frames = get_pos_alt(curr_tag)

    ax.scatter(x, y, z, color=c, label=str(curr_tag))
    plt.legend(bbox_to_anchor=(1.2, 1), loc="upper left")
plt.show()

In [ ]:
fig.savefig(artag_dir + artag_filename + 'sroa_tags.png')

In [ ]:
# Plot positions of ALL detected tags (OSA & SROA)
fig = plt.figure(figsize=(10, 8))

ax = plt.axes(projection='3d')
ax.set_title('Position of OSA vs SROA Tags')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Postion')

for i in range(len(all_tags)):
    curr_tag = all_tags[i]

    x, y, z, frames = get_pos_alt(curr_tag)

    if curr_tag < 110:
      ax.scatter(x, y, z, color='blue', label=str(curr_tag))
    else:
      ax.scatter(x, y, z, color='red', label=str(curr_tag))

    plt.legend(bbox_to_anchor=(1.2, 1), loc="upper left")
# plt.legend(['OSA', 'SROA'], loc="upper right")
plt.show()

In [ ]:
fig.savefig(artag_dir + artag_filename + 'sroa_vs_osa_tags.png')

In [ ]:
# Plot position of SOME detected tags (OSA & SROA)
fig = plt.figure(figsize=(8, 8))

# TO DO: Need to set the amount of tags to plot
num_tags = 5

# Set color range
colors = cm.rainbow(np.linspace(0, 1, len(all_tags[:num_tags])))
ax = plt.axes(projection='3d')
ax.set_title('Position of ' + str(num_tags) + ' Detected Tags')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Postion')

for i, c in zip(range(len(all_tags[:num_tags])), colors):
    curr_tag = all_tags[i]#+28]
    x, y, z, frames = get_pos_alt(curr_tag)
    ax.scatter(x, y, z, color=c, label=str(curr_tag))
    plt.legend(bbox_to_anchor=(1.2, 1), loc="upper left")

plt.show()

In [ ]:
fig.savefig(artag_dir + artag_filename + str(num_tags) +'_tags.png')

In [ ]:
# Plot position of SOME detected tags (OSA & SROA)
fig = plt.figure(figsize=(8, 8))

# print(all_tags)
sroas =[]
osas = []
t13_14 =[]
for t in all_tags:
  if t in sroa:
      # print("[INFO] Tag {} is in SROA".format(str(r.tag_id)))
      sroas.append(t)
      tag = "SROA"
  elif t not in sroa:
      # print("[INFO] Tag {} is in OSA".format(str(r.tag_id)))
      osas.append(t)
      tag = "OSA"
  if t == 13 or t == 14:
    t13_14.append(t)

print('SROA: ')
print(sroas)
print('OSA: ')
print(osas)
print('all tags: ')
print(str(len(all_tags)))
num_tags = len(sroas)
# print(num_tags)

# Set color range
colors = cm.rainbow(np.linspace(0, 1, len(all_tags[:num_tags])))
ax = plt.axes(projection='3d')
ax.set_title('Position of ' + str(num_tags) + ' SROA Detected Tags')
ax.set_xlabel('X Position')
ax.set_ylabel('Y Position')
ax.set_zlabel('Z Postion')

for i, c in zip(range(len(all_tags[:num_tags])), colors):
    curr_tag = all_tags[i]
    x, y, z, frames = get_pos_alt(curr_tag)
    ax.scatter(x, y, z, color=c, label=str(curr_tag))
    plt.legend(bbox_to_anchor=(1.2, 1), loc="upper left")

plt.show()

In [ ]:
fig.savefig(artag_dir + artag_filename + str(num_tags) +'_osa_tags.png')

#Part 3
*Find the distance between OSA tags by frame and by magnet, then compare it to the distances recorded in the metrology*

In [ ]:
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
# Returns list of all detected tags, duplicates not allowed and amount of frames
def get_logistics():
  tags = []
  frames = []
  with open(filename, newline='') as csvfile:
          reader = csv.DictReader(csvfile)
          for row in reader:
                  tags.append(int(row['tag ID']))
                  frames.append(int(row['frame']))

  tags = [*set(tags)]
  frames = [*set(frames)]
  return tags, frames

tags, frames = get_logistics()
print("Amount of unique tags: " + str(len(tags)))
print(tags)
print("Amount of unique frames: " + str(len(frames)))
print(frames)

In [ ]:
# Open csv file sort, get all tags by tagID < 110
# Keys: frame
# Values: tags
def get_osa_tags_dict():
  osa_tags = defaultdict(list)

  with open(filename, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      # Get the tags on the OSA. OSA tags values do not go over 110. See metrology.
      if int(row['tag ID']) < 110:
        frame = row['frame']
        tag = row['tag ID']
        osa_tags[frame].append(tag)

  return osa_tags

# Dictionary of tags per frame
osa_tags2 = get_osa_tags_dict()
print(osa_tags2)
# print(osa_tags.keys())
# print(osa_tags.values())

In [ ]:
# Find all possiible detected pairs on the same magnet in the same frame
for frame in osa_tags2.keys():
  curr_tags = osa_tags2[frame]
  print("Frame: " + frame)
  for i in range(len(curr_tags)):
    for j in range(i + 1, len(curr_tags)):
      curr = int(curr_tags[i])
      next = int(curr_tags[j])

      # Compare each tag within a frame, plus or minus 4, to find if it is on the same panel of OSA
      if (abs(curr - next) < 5):
        print("Match found!")

In [ ]:
# Once a pair is found, open up the data again.
# Find the correct data that corresponds to the tag and frame then find distance
def get_displacement(osa_tags2):
  result = []
  pairs = []
  for frame in osa_tags2.keys():
    # print(osa_tags2[frame])
    curr_tags = osa_tags2[frame]
    if (len(curr_tags) > 1):
      print("\nFrame: " + frame)
      for i in range(len(curr_tags)):
        for j in range(i + 1, len(curr_tags)):
          curr = int(curr_tags[i])
          next = int(curr_tags[j])

          # Compare each tag within a frame, plus or minus 4, to find if it is on the same panel of OSA
          if (abs(curr - next) < 5):
            # print("Match found!")
            with open(filename, newline='') as csvfile:
              reader = csv.DictReader(csvfile)
              x_1 = 0
              y_1 = 0
              z_1 = 0
              x_2 = 0
              y_2 = 0
              z_2 = 0
              disp = 0

              # per frame, a unique tag can only be detected once
              for row in reader:
                if (row['frame'] == str(frame) and row['tag ID'] == str(curr)):
                  x_1 = float(row['pos x'].strip('[]'))
                  y_1 = float(row['pos y'].strip('[]'))
                  z_1 = float(row['pos z'].strip('[]'))

                if (row['frame'] == str(frame) and row['tag ID'] == str(next)):
                  x_2 = float(row['pos x'].strip('[]'))
                  y_2 = float(row['pos y'].strip('[]'))
                  z_2 = float(row['pos z'].strip('[]'))

              disp = math.sqrt((x_1 - x_2)**2 + (y_1 - y_2)**2 + (z_1 - z_2)**2)
              curr_result = [frame, curr, next, disp]
              result.append(curr_result)
              if [curr, next] not in pairs:
                pairs.append([curr, next])
              print("Displacement between tag " + str(curr) + " and tag " + str(next) + ": " + str(disp) + "mm")
  return result, pairs

In [ ]:
# result holds the frame, pair tag, and the dispplacement between the tags at that frame
# pairs holds all posible pairs detected
result, pairs = get_displacement(osa_tags2)

In [ ]:
# Display the return from get_displacement() function
# result will display the frame, the pair tag, and the distance between the pair tags
# pairs will display all possible pairs detected
print(result)
print(pairs)

In [ ]:
# Use result from get_displacement() function and place into a new csv file
# columns: frame, curr, next, displacement
def displacement_csv(result):
  with open (filename + '_displacement.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Frame", "Tag 1", "Tag 2", "Displacement"])
    for row in result:
      writer.writerow(row)

In [ ]:
displacement_csv(result)

### TO DO: Choose a pair of detected tags from the list of tag pairs below

In [ ]:
# List all pairs of detected tags
pairs.sort()
for pair in pairs:
  print(pair)

# TO DO: Choose a pair of tags from the list above

In [ ]:
# TO DO: Choose a pair of tags
tag1 = 13
tag2 = 14

In [ ]:
# Check if it's valid
def check_tag_pairs(tag1, tag2):
  if tag1 == tag2:
    print('Error. Please choose unique tags for a tag pair.')
  elif (abs(tag2 - tag1) > 4):
    print('Error. Please choose tags from the same group.')
  else:
    if [tag1, tag2] in pairs:
      print('Comparing tags ' + str(tag1) + ' and ' + str(tag2))
      return True
    else:
      print('Tag pair not detected. Please choose a pair from the list.')
  return False

In [ ]:
check_tag_pairs(tag1, tag2)

In [ ]:
# Open displacement csv
# Collect data for the specified valid tag pair
# This data will be used to calculate mean squared error
def get_data_tag_pair(filename, tag1, tag2):
  dist_result = []
  frames_result = []

  with open (filename + '_displacement.csv', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
      if (row['Tag 1'] == str(tag1) and row['Tag 2'] == str(tag2)):
        dist_result.append([row])
        frames_result.append(row['Frame'])

  return dist_result, frames_result

In [ ]:
dist_result, frames_result = get_data_tag_pair(filename, tag1, tag2)

In [ ]:
def plot_dist_tag_pair(frames, frames_result, dist_result, tag1, tag2):
  dist = []
  index = 0

  for frame in frames:
    if str(frame) in frames_result:
      dist.append(float(dist_result[index][0]['Displacement']))
      index += 1
    else:
      dist.append(np.nan)

  # determine the expected distance between tag pair
  # if tags are next to each other, they are 39mm apart, else 64mm apart
  exp_dist = 0
  if (abs(tag1 - tag2) == 1 or abs(tag1 - tag2) == 4):
    exp_dist = 39
  else:
    exp_dist = 64

  legend_label_pairs = 'Tag ' + str(tag1) + ' & ' + str(tag2)
  legend_label_exp = 'Expected Distance: ' + str(exp_dist)
  plt.plot(frames, dist, label=legend_label_pairs)
  plt.plot(frames, [exp_dist]*len(dist), label=legend_label_exp)
  plt.xlabel('Frame')
  plt.ylabel('Distance between tag pair')
  plt.legend()

  # save the plot
  plt.savefig(artag_dir + artag_filename + 'Measured & Expected Distance (' + str(tag1) + ', ' + str(tag2) + ').png')

  plt.show()
  return dist, exp_dist

In [ ]:
dist, exp_dist = plot_dist_tag_pair(frames, frames_result, dist_result, tag1, tag2)

In [ ]:
# Calculate the MSE between the measured distance and the actual distance of tag pair
def get_mse(test, pred, tag1, tag2):
  from sklearn.metrics import mean_squared_error

  # y_test are the expected displacement values (39mm or 64mm)
  y_test = [x for x in test if str(x) != "nan"]

  # y_pred are the displacement values that were detected and calculated
  y_pred = [pred]*len(y_test)

  mse = mean_squared_error(y_test, y_pred)
  print('Tag pair: ' + str(tag1) + ', ' + str(tag2))
  print('Mean Squared Error', mse)

  return mse

In [ ]:
mse = get_mse(dist, exp_dist, tag1, tag2)

#Use the functions above to plot graphs based on displacement

In [ ]:
# Make plots for all tag pairs
all_mse = []

for pair in pairs:
  tag1 = pair[0]
  tag2 = pair[1]

  # Open displacement CSV and get data on selected tag pair
  dist_result, frames_result = get_data_tag_pair(filename, tag1, tag2)

  # Plot the tag pair
  dist, exp_dist = plot_dist_tag_pair(frames, frames_result, dist_result, tag1, tag2)

  # Print the MSE for the tag pair
  mse = get_mse(dist, exp_dist, tag1, tag2)
  all_mse.append(float(mse))

In [ ]:
# Create CSV with MSE (optional)
results = []
index = 0

for pair in pairs:
  tag1 = pair[0]
  tag2 = pair[1]

  exp_dist = 0
  if (abs(tag1 - tag2) == 1 or abs(tag1 - tag2) == 4):
    exp_dist = 39
  else:
    exp_dist = 64

  results.append([tag1, tag2, exp_dist, all_mse[index]])

  index += 1

with open ('MSE.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerow(["Tag 1", "Tag 2", "Expected", "MSE"])
  for row in results:
      writer.writerow(row)

#End of part 3

## PART 4

> Documentation



In [ ]:
document = Document()

document.add_heading(artag_filename, 0)

# TO DO: Change Quad countour's value, units and tag size
document.add_paragraph('File Name: ' + filename, style='List Bullet')
document.add_paragraph('Radius: ' + str(radius) + ' mm', style='List Bullet')
#document.add_paragraph('F: ' + str(f) + ' mm', style='List Bullet')
document.add_paragraph('Total No. of Frames: ' + str(total_num_frame) + 'frames', style='List Bullet')
document.add_paragraph('F: (' + str(fx) + ', ' + str(fy) + ') ', style='List Bullet')
document.add_paragraph('C: (' + str(cx) + ', ' + str(cy) + ') ', style='List Bullet')
document.add_paragraph('Dimensions: (' + str(original_width) + ', ' + str(original_height) + ') ', style='List Bullet')
document.add_paragraph('Tag Size: 14 mm (real)', style='List Bullet')
document.add_paragraph('Scaled: (' + str(scale_x) + ', ' + str(scale_y) + ') ', style='List Bullet')
document.add_paragraph('Camera Parameters: ' + str(camera_params), style='List Bullet')
document.add_paragraph('Date: ' + str(date), style='List Bullet')
document.add_paragraph('Video Name: ' + videoname, style='List Bullet')
document.add_paragraph('FPS: ' + str(fps), style='List Bullet')
document.add_paragraph('Quad_contours: True', style='List Bullet')
document.add_paragraph('Total number of tags: ' + str(len(osas)) + ' + ' + str(len(sroas)), style='List Bullet')
document.add_paragraph('OSA: ' + str(osas), style='List Bullet')
document.add_paragraph('SROA: ' + str(sroas), style='List Bullet')
document.add_paragraph('options = apriltag.DetectorOptions(families="tag36h11",border =1, nthreads=4, quad_decimate =1.0,quad_blur=0.0, refine_edges=True, refine_decode=False,refine_pose=True, debug=True, quad_contours=True)', style='List Bullet')
document.add_picture(artag_dir + artag_filename + 'all_tags.png', width=Inches(5), height=Inches(5))
document.add_picture(artag_dir + artag_filename + str(num_tags) +'_osa_tags.png', width=Inches(5), height=Inches(5))
document.add_picture('Measured & Expected Distance (' + str(tag1) + ', ' + str(tag2) + ').png' , width=Inches(5), height=Inches(5))

document.save(artag_dir + artag_filename + '.docx')
print(artag_dir)


In [ ]:
# doc_folder = '/content/drive/MyDrive/AR Tags/2. Data/Code Data/'
# print(doc_folder)
# doc_filename = artag_filename + '_Code'
# print(doc_filename)

# Define the folder path where you want to save the Google Doc
doc_folder = artag_dir
print(doc_folder)

# Define the file name for the Google Doc
doc_filename = artag_filename + '_Code'

# Path of the current Colab notebook
notebook_path = '/content/drive/MyDrive/AR Tags/5. Code/AprilTagDetection.ipynb'

In [ ]:
import pprint
import pygments
import webbrowser
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter

# Read the content of the notebook file
with open(notebook_path, 'r') as f:
    code_content = f.read()

# Pretty-print the code content
pretty_code = pprint.pformat(code_content)

# Define the path and filename for the Google Doc
doc_path = doc_folder + doc_filename + '.docx'

# Save the code content as a document file
with open(doc_path, 'w') as f:
    f.write(code_content)


print("Code saved successfully as a document file in the desired folder.")


In [ ]:
# # Read the content of the notebook file
# with open(notebook_path, 'r') as f:
#     code_content = f.read()

# # Specify the programming language for syntax highlighting (e.g., 'python', 'java', 'javascript')
# language = 'python'

# # Get the lexer for the specified programming language
# lexer = get_lexer_by_name(language)

# # Apply syntax highlighting to the code content
# highlighted_code = pygments.highlight(code_content, lexer, HtmlFormatter())

# # Define the path and filename for the HTML file
# html_path = doc_folder + doc_filename + '.html'

# # Save the highlighted code as an HTML file
# with open(html_path, 'w') as f:
#     f.write(highlighted_code)

# # Open the HTML file in a web browser
# webbrowser.open('file://' + os.path.abspath(html_path))

# print("Code saved successfully as an HTML file in the desired folder.")


In [ ]:
# import requests

# # Read the content of the notebook file
# with open(notebook_path, 'r') as f:
#     code_content = f.read()

# # Generate a new Google Doc
# gdoc_url = 'https://docs.google.com/document/create'
# response = requests.get(gdoc_url)
# response.raise_for_status()
# gdoc_url = response.url

# # Open the Google Doc in edit mode
# edit_url = gdoc_url + '/edit'
# response = requests.get(edit_url)
# response.raise_for_status()

# # Extract the new Google Doc ID from the response URL
# new_doc_id = response.url.split('/')[-2]
# #
# # Make a POST request to the document's edit URL to set the document content
# post_url = f'https://docs.google.com/document/d/{new_doc_id}/edit'
# data = {'content': code_content}
# response = requests.post(post_url, data=data)
# response.raise_for_status()

# print("Code copied and saved to a newly generated Google Doc.")

# # Move the new Google Doc to the desired folder
# folder_id = '14ubGTrSh-HS_u1PfIgXOF_BpM8nKo_nj'  # Replace with the actual folder ID
# move_doc_url = f'https://docs.google.com/document/d/{new_doc_id}/moveTo/folder/{folder_id}'
# requests.get(move_doc_url)

# print("Code copied and saved to a Google Doc in the specified folder.")


In [ ]:

# # Read the content of the notebook file
# with open(notebook_path, 'r') as f:
#     code_content = f.read()

# # # Pretty-print the code content
# pretty_code = pprint.pformat(code_content)

# # Set the destination folder path
# destination_folder = doc_folder

# # Set the destination file name
# destination_filename = doc_filename + '.txt'

# # Save the code as a plain text file
# destination_path = destination_folder + destination_filename
# with open(destination_path, 'w') as f:
#     f.write(code_content)

# print("Code copied and saved to a text file in the specified folder.")

## Part 5: Linear Regression and difference